# <a id='model-selection-and-evaluation'></a> 3. [**Sélection et évaluation de modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#model-selection-and-evaluation)</br>([*Model selection and evaluation*](https://scikit-learn.org/stable/model_selection.html#model-selection-and-evaluation))

# 3.1. [**Validation croisée : évaluer les performances des estimateurs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-evaluating-estimator-performance)<br/>([_Cross-validation: evaluating estimator performance_](https://scikit-learn.org/stable/model_selection.html#cross-validation-evaluating-estimator-performance))

# Sommaire

- **Volume** : 25 pages, 7 exemples, 7 papiers
- 3.1.1. [**Calcul de métriques à validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#computing-cross-validated-metrics)<br/>([_computing-cross-validated-metrics_](https://scikit-learn.org/stable/model_selection.html#computing-cross-validated-metrics))
- 3.1.2. [**Itérateurs de validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-iterators)<br/>([_Cross validation iterators_](https://scikit-learn.org/stable/model_selection.html#cross-validation-iterators))
- 3.1.3. [**Une note sur le brassage**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#a-note-on-shuffling)<br/>([_A note on shuffling_](https://scikit-learn.org/stable/model_selection.html#a-note-on-shuffling))
- 3.1.4. [**Validation croisée et sélection de modèles**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-and-model-selection)<br/>([_cross-validation-and-model-selection_](https://scikit-learn.org/stable/model_selection.html#cross-validation-and-model-selection))
- 3.1.5. [**Résultat du test de permutation**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#permutation-test-score)<br/>([_Permutation test score_](https://scikit-learn.org/stable/model_selection.html#permutation-test-score))

# <a id='computing-cross-validated-metrics'></a> 3.1. Validation croisée : évaluer les performances des estimateurs

Apprendre les paramètres d'une fonction de prédiction et la tester sur les mêmes données est une erreur méthodologique : un modèle qui se contenterait de répéter les étiquettes des échantillons qu'il vient de voir aurait un score parfait mais ne parviendrait pas à prédire quoi que ce soit d'utile sur de nouvelles données. Cette situation s'appelle le **surajustement**. Pour l'éviter, il est courant lors de la réalisation d'une expérience d'apprentissage automatique (supervisée) de conserver une partie des données disponibles sous forme d'**ensemble de test** `X_test`, `y_test`. Notez que le mot "expérience" n'est pas destiné à désigner uniquement un usage académique, car même dans les environnements commerciaux, l'apprentissage automatique commence généralement de manière expérimentale. Voici un organigramme du flux de travail typique de validation croisée dans l'entraînement de modèles. Les meilleurs paramètres peuvent être déterminés par des techniques comme la [**recherche en grille** (3.2.1)](https://scikit-learn.org/stable/modules/grid_search.html#grid-search).


<div style="background-color: white; text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/grid_search_workflow.png"
    alt="Flux de travail de recherche en grille"
    style="max-width: 50%; height: auto;">
</div>

Dans scikit-learn, une répartition aléatoire en ensembles d'apprentissage et de test peut être rapidement calculée avec la fonction utilitaire [**`train_test_split`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split). Chargeons l'ensemble de données Iris pour y adapter une machine à vecteurs de support linéaire :

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

((150, 4), (150,))

Nous pouvons désormais échantillonner rapidement un ensemble d'entraînement tout en conservant 40 % des données pour tester (évaluer) notre classifieur :

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

X_train.shape, y_train.shape
# ((90, 4), (90,))
X_test.shape, y_test.shape
# ((60, 4), (60,))

clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)
# 0.96...

0.9666666666666667

Lors de l'évaluation de différents paramétrages ("hyperparamètres") pour les estimateurs, tels que le paramètre `C` qui doit être défini manuellement pour une SVM, il existe toujours un risque de surapprentissage sur l'_ensemble de test_ car les paramètres peuvent être modifiés jusqu'à ce que l'estimateur fonctionne de manière optimale. De cette façon, les connaissances sur l'ensemble de test peuvent "fuir" dans le modèle et les métriques d'évaluation ne plus rendre compte des performances de généralisation. Pour résoudre ce problème, une autre partie de l'ensemble de données peut être isolée en tant qu'"ensemble de validation": l'entraînement se déroule sur l'ensemble d'entraînement, après quoi l'évaluation est effectuée sur l'ensemble de validation, et quand l'expérience semble être réussie, l'évaluation finale peut être effectuée sur l'ensemble de test.

Cependant, en partitionnant les données disponibles en trois ensembles, nous réduisons considérablement le nombre d'échantillons qui peuvent être utilisés pour entraîner le modèle, et les résultats peuvent dépendre d'un choix aléatoire particulier pour la paire d'ensembles (entraînement, validation).

Une solution à ce problème est une procédure appelée [⦿ **validation croisée**](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) (CV (_Cross validation_) en abrégé). Un ensemble de test doit toujours être conservé pour l'évaluation finale, mais l'ensemble de validation n'est plus nécessaire lors de la réalisation de la CV. Dans l'approche de base, appelée $k$-fold CV, l'ensemble d'apprentissage est divisé en $k$ ensembles plus petits (d'autres approches sont décrites ci-dessous, mais suivent généralement les mêmes principes). La procédure suivante est suivie pour chacun des $k$ "plis" :

- Un modèle est entraîné sur $k - 1$ plis en tant que données d'apprentissage ;
- le modèle résultant est validé sur le pli restant (il est utilisé comme ensemble de test pour calculer une mesure de performance telle que l'exactitude).

La mesure de performance rapportée par la validation croisée $k$-plis est alors la moyenne des valeurs calculées dans la boucle. Cette approche peut être coûteuse en calcul, mais ne gaspille pas trop de données (comme c'est le cas lors du choix d'un ensemble de validation arbitraire), ce qui est un avantage majeur dans des problèmes tels que l'inférence inverse où le nombre d'échantillons est très petit.

<div style="background-color: white; text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png"
    alt="Flux de travail de recherche en grille"
    style="max-width: 50%; height: auto;">
</div>

## <a id='computing-cross-validated-metrics'></a> 3.1.1. Calcul de métriques à validation croisée

La manière la plus simple d'utiliser la validation croisée consiste à appeler la fonction utilitaire [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) sur l'estimateur et l'ensemble de données.

L'exemple suivant montre comment estimer la précision d'une machine à vecteurs de support à noyau linéaire sur l'ensemble de données iris en divisant les données, en ajustant un modèle et en calculant le score 5 fois consécutives (avec des divisions différentes à chaque fois) :

In [3]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
scores
# array([0.96..., 1. , 0.96..., 0.96..., 1. ])

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

Le score moyen et l'écart type sont alors donnés par :

In [4]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
# 0.98 accuracy with a standard deviation of 0.02

0.98 accuracy with a standard deviation of 0.02


Par défaut, le score calculé à chaque itération de la CV est donné par la méthode `score` de l'estimateur. Il est possible de changer cela en utilisant le paramètre `scoring`` :

In [ ]:
from sklearn import metrics
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
scores
# array([0.96..., 1.  ..., 0.96..., 0.96..., 1.        ])

array([0.96658312, 1.        , 0.96658312, 0.96658312, 1.        ])

Voir [**Le paramètre `scoring` : définir les règles d'évaluation du modèle** (3.3.1)](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) pour plus de détails. Dans le cas de l'ensemble de données Iris, les échantillons sont équilibrés entre les classes cibles, d'où le fait que l'exactitude et le score F1 sont presque égaux.

Lorsque l'argument `cv` est un entier, [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) utilise par défaut les stratégies [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) ou [**`StratifiedKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html), cette dernière étant utilisée si l'estimateur dérive de [**`ClassifierMixin`**](https://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html).

Il est également possible d'utiliser d'autres stratégies de validation croisée en passant un itérateur de validation croisée à la place, par exemple :

In [5]:
from sklearn.model_selection import ShuffleSplit
n_samples = X.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(clf, X, y, cv=cv)
# array([0.977..., 0.977..., 1.  ..., 0.955..., 1.        ])

array([0.97777778, 0.97777778, 1.        , 0.95555556, 1.        ])

Une autre option consiste à utiliser un itérable fournissant des séparations (entraînement, test) sous forme de tableaux d'indices, par exemple :

In [ ]:
def custom_cv_2folds(X):
    n = X.shape[0]
    for i in range(1, 3):
        idx = np.arange(n * (i - 1) / 2, n * i / 2, dtype=int)
        yield idx, idx

custom_cv = custom_cv_2folds(X)
cross_val_score(clf, X, y, cv=custom_cv)
# array([1.        , 0.973...])

array([1.        , 0.97333333])

**Transformation de données avec des données conservées**

Tout comme il est important de tester un estimateur sur les données d'entraînement conservées, le prétraitement (comme la normalisation, la sélection de caractéristiques, etc.) et les [**Transformations de données** (6)](https://scikit-learn.org/stable/data_transforms.html#data-transforms) similaires doivent également être apprises à partir d'un ensemble d'entraînement puis appliquées aux données conservées pour la prédiction :

In [6]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = svm.SVC(C=1).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
clf.score(X_test_transformed, y_test)
# 0.9333...

0.9333333333333333

Un [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) facilite la composition des estimateurs, offrant ce comportement sous validation croisée :

In [8]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))
cross_val_score(clf, X, y, cv=cv)
# array([0.977..., 0.933..., 0.955..., 0.933..., 0.977...])

array([0.97777778, 0.93333333, 0.95555556, 0.93333333, 0.97777778])

Voir [**Pipelines et estimateurs composites** (6.1)](https://scikit-learn.org/stable/modules/compose.html#combining-estimators).

### <a id='the-cross-validate-function-and-multiple-metric-evaluation'></a> 3.1.1.1. La fonction `cross_validate` et l'évaluation de métriques multiples

La fonction [**`cross_validate`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) diffère de [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) de deux manières :

- Elle permet de spécifier plusieurs métriques pour l'évaluation.
- Elle renvoie un dictionnaire contenant les temps d'ajustement, les temps d'évaluation (et éventuellement les scores d'entraînement, les estimateurs ajustés, les indices de la partition entraînement-test) en plus du score du test.

Pour une évaluation à métrique unique, où le paramètre de score est une chaîne, un `callable` ou `None`, les clés seront - `['test_score', 'fit_time', 'score_time']`

Et pour l'évaluation à métriques multiples, la valeur de retour est un dictionnaire avec les clés suivantes - `['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']`

La valeur par défaut de `return_train_score` est `False` pour économiser du temps de calcul. Pour évaluer également les scores sur l'ensemble d'entraînement, vous devez le définir sur `True`. Vous pouvez également conserver l'estimateur ajusté pour chaque ensemble d'entraînement en définissant `return_estimator=True`. De même, vous pouvez définir `return_indices=True` pour conserver les indices d'entraînement et de test utilisés pour diviser le jeu de données en ensembles d'entraînement et de test pour chaque partition CV.

Les multiples métriques peuvent être spécifiées sous forme de liste, de tuple ou d'ensemble de noms d'évaluateurs prédéfinis :

In [9]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, X, y, scoring=scoring)
sorted(scores.keys())
# ['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']
scores['test_recall_macro']
# array([0.96..., 1.  ..., 0.96..., 0.96..., 1.        ])

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

Ou sous la forme d'un dictionnaire qui fait correspondre le nom de l'évaluateur à une fonction d'évaluation prédéfinie ou personnalisée :

In [10]:
from sklearn.metrics import make_scorer
scoring = {
    'prec_macro': 'precision_macro',
    'rec_macro': make_scorer(recall_score, average='macro')
}
scores = cross_validate(clf, X, y, scoring=scoring, cv=5, return_train_score=True)
sorted(scores.keys())
# ['fit_time', 'score_time', 'test_prec_macro', 'test_rec_macro',
#  'train_prec_macro', 'train_rec_macro']
scores['train_rec_macro']
# array([0.97..., 0.97..., 0.99..., 0.98..., 0.98...])

array([0.975     , 0.975     , 0.99166667, 0.98333333, 0.98333333])

Voici un exemple de `cross_validate` utilisant une métrique unique :

In [11]:
scores = cross_validate(clf, X, y,
                        scoring='precision_macro', cv=5,
                        return_estimator=True)
sorted(scores.keys())
# ['estimator', 'fit_time', 'score_time', 'test_score']

['estimator', 'fit_time', 'score_time', 'test_score']

### <a id='obtaining-predictions-by-cross-validation'></a> 3.1.1.2. Obtenir des prédictions par validation croisée

La fonction [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) a une interface similaire à [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), mais renvoie, pour chaque élément de l'entrée, la prédiction qui a été obtenue pour cet élément lorsqu'il était dans l'ensemble de test. Seules les stratégies de validation croisée qui affectent tous les éléments à un ensemble de test exactement une fois peuvent être utilisées (sinon, une exception est levée).

> **Avertissement** : Remarque sur l'utilisation inappropriée de [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)  
> Le résultat de [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) peut être différent de ceux obtenus avec [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) car les éléments sont regroupés différemment. La fonction [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) prend une moyenne sur les plis de validation croisée, tandis que [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) renvoie simplement les étiquettes (ou probabilités) de plusieurs modèles distincts non distingués. Ainsi, [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) n'est pas une mesure appropriée de l'erreur de généralisation.

**La fonction [`cross_val_predict`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html#sklearn.model_selection.cross_val_predict) convient pour :**

- La visualisation des prédictions obtenues à partir de différents modèles.
- Le mélange de modèles : lorsque les prédictions d'un estimateur supervisé sont utilisées pour entraîner un autre estimateur dans le cadre des méthodes ensemblistes.

Les itérateurs de validation croisée disponibles sont présentés dans la section suivante.

## Exemples

1. Receiver Operating Characteristic (ROC) avec validation croisée,
2. Élimination récursive des caractéristiques avec validation croisée,
3. Stratégie de refit personnalisée d'une grille de recherche avec validation croisée,
4. Exemple de pipeline pour l'extraction et l'évaluation de caractéristiques de texte,
5. Tracer des prédictions validées croisées,
6. Validation croisée imbriquée versus non imbriquée.